In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import json
import re

In [84]:
class request():
    def __init__(self):
        self.Name = ""
        self.Ranking = 0
        self.Surname = ""
        self.Given_name = ""
        self.Total_score = 0
        
        self.School_score = 0
        self.Gov_exams = {}
        self.Univ_exams = {}
        self.Extra_points = {}
        
        self.Is_out_of_competition = False
        #for comp. with 2014 data
        self.Is_prioritized = False
        #for comp. with 2014 data
        self.Is_directed = False
        
        self.Is_original = False
        self.Is_enrolled = False
        self.Priority = 0
        self.Status = ""

In [85]:
subj_mapping = {"Математика" : "math", "Українська мова та література" : "ukr",
                "Іноземна мова" : "foreign", "Фізика" : "phys", "Хімія" : "chem", 
                "Біологія" : "bio", "Географія" : "geo", "Англійська мова" : "eng", 
                "Історія України" : "ukr_hist", "Фахове випробування" : "profile_exam", "Російська мова" : "rus",
                "Творчий конкурс" : "art"}

In [86]:
def parse_details(src):
    school_score = 0
    try:
        school_score = float(re.search("Середній бал документа про освіту: ([0-9]*\.[0-9]+|[0-9]+)", src).group(1))
    except AttributeError:
        pass
    extra_points = {}
    try:
        extra_points["Olimpiads"] = int(re.search("Бал за особливі успіхи : ([0-9]*\.[0-9]+|[0-9]+)", src).group(1))
    except AttributeError:
        pass
    gov_exams = {}
    for item in subj_mapping.keys():
        try:
            gov_exams[subj_mapping[item]] = int(re.search(str(item) + " \(ЗНО\): ([0-9]*)", src).group(1))
        except AttributeError:
            pass
    univ_exams = {}
    for item in subj_mapping.keys():
        try:
            univ_exams[subj_mapping[item]] = int(re.search(str(item) + " \(іспит\): ([0-9]*)", src).group(1))
        except AttributeError:
            pass
        try:
            univ_exams[subj_mapping[item]] = int(re.search(str(item) + ": ([0-9]*)", src).group(1))
        except AttributeError:
            pass
    return school_score, gov_exams, univ_exams, extra_points

In [87]:
src = "Іноземна мова (ЗНО): 167 Історія України (ЗНО): 157 Українська мова та література (ЗНО): 178 Середній бал документа про освіту: 10.1"

In [88]:
parse_details(src)

(10.1, {'ukr': 178, 'foreign': 167, 'ukr_hist': 157}, {}, {})

In [153]:
def html_2018_to_parsed_json(src):
    soup = bs(src,'html.parser')
    tab=soup.find("table", {"class":"tablesaw tablesaw-stack tablesaw-sortable"})
    df = pd.read_html(str(tab), skiprows=0)[0]
    res = []
    for index, row in df.iterrows():
        r = request()
        r.Name, r.Surname, r.Given_name = (row['ПІБ'] + ' - - ').split(' ')[:3]
        r.Total_score = row['Σ']
        r.Priority = row['П']
        r.Status = str(row['С'])
        r.Ranking = row['#']
        r.Is_enrolled = (str(row["С"])[:9] == "До наказу")
        r.Is_out_of_competition = (str(row['К']) != "—")
        r.Is_original = (str(row['Д']) == "+")
        r.School_score, r.Gov_exams, r.Univ_exams, r.Extra_points = parse_details(row['Деталізація'])
        res.append(json.dumps(r.__dict__))
    return res

In [154]:
url='http://vstup.info/2018/41/i2018i41p449176.html'
r=requests.get(url)
url=r.content

In [155]:
r = request()

In [156]:
json.dumps(r.__dict__)

'{"Name": "", "Ranking": 0, "Surname": "", "Given_name": "", "Total_score": 0, "School_score": 0, "Gov_exams": {}, "Univ_exams": {}, "Extra_points": {}, "Is_out_of_competition": false, "Is_prioritized": false, "Is_directed": false, "Is_original": false, "Is_enrolled": false, "Priority": 0, "Status": ""}'

In [157]:
jsn = html_2018_to_parsed_json(url)

In [158]:
jsn

['{"Name": "\\u041a\\u043e\\u043b\\u0454\\u0441\\u043d\\u0456\\u043a", "Ranking": "1", "Surname": "\\u041e.", "Given_name": "\\u041e.", "Total_score": "200.000", "School_score": 11.0, "Gov_exams": {"math": 189, "ukr": 184, "geo": 191}, "Univ_exams": {}, "Extra_points": {}, "Is_out_of_competition": false, "Is_prioritized": false, "Is_directed": false, "Is_original": true, "Is_enrolled": true, "Priority": "1", "Status": "\\u0414\\u043e \\u043d\\u0430\\u043a\\u0430\\u0437\\u0443 (\\u0431\\u044e\\u0434\\u0436\\u0435\\u0442)"}',
 '{"Name": "\\u041a\\u043e\\u043b\\u0435\\u0441\\u043d\\u0438\\u043a", "Ranking": "2", "Surname": "\\u0410.", "Given_name": "\\u042e.", "Total_score": "193.902", "School_score": 11.1, "Gov_exams": {"ukr": 186, "geo": 191, "ukr_hist": 194}, "Univ_exams": {}, "Extra_points": {}, "Is_out_of_competition": false, "Is_prioritized": false, "Is_directed": false, "Is_original": true, "Is_enrolled": true, "Priority": "1", "Status": "\\u0414\\u043e \\u043d\\u0430\\u043a\\u0430

In [67]:
df1 = pd.read_json(jsn)

In [68]:
df1.sort_index()

,#,ПІБ,П,Σ,С,Деталізація,К,Д
0,1,Колєснік О. О.,1,200.000,До наказу (бюджет),Українська мова та література (ЗНО): 184 Геогр...,—,+
1,2,Колесник А. Ю.,1,193.902,До наказу (бюджет),Українська мова та література (ЗНО): 186 Геогр...,—,+
2,3,Годунко С. Ю.,1,188.468,До наказу (бюджет),Українська мова та література (ЗНО): 184 Геогр...,—,+
3,4,Ставничий Н. Т.,2,187.476,До наказу (бюджет),Українська мова та література (ЗНО): 168 Геогр...,—,+
4,5,Колодежна В. В.,1,187.221,До наказу (бюджет),Українська мова та література (ЗНО): 183 Геогр...,—,+
5,6,Далудіна Д. В.,2,187.119,До наказу (бюджет),Українська мова та література (ЗНО): 185 Геогр...,—,+
6,7,Черемшук С. С.,1,186.507,До наказу (бюджет),Українська мова та література (ЗНО): 191 Геогр...,—,+
7,8,Ольхова І. Е.,1,185.946,До наказу (бюджет),Українська мова та література (ЗНО): 188 Геогр...,—,+
8,9,Трохимець С. О.,1,185.640,До наказу (бюджет),Українська мова та література (ЗНО): 189 Геогр...,—,+
9,10,Шопша Ю. А.,1,185.640,До наказу (бюджет),Українська мова та література (ЗНО): 190 Геогр...,—,+


In [37]:
soup = bs(url,'html.parser')
tab=soup.find("table", {"class":"tablesaw tablesaw-stack tablesaw-sortable"})

In [38]:
ldf = pd.read_html(str(tab), skiprows=0)

In [41]:
df = ldf[0]

In [45]:
df.set_index('#')

,ПІБ,П,Σ,С,Деталізація,К,Д
#,,,,,,,
1,Колєснік О. О.,1,200.000,До наказу (бюджет),Українська мова та література (ЗНО): 184 Геогр...,—,+
2,Колесник А. Ю.,1,193.902,До наказу (бюджет),Українська мова та література (ЗНО): 186 Геогр...,—,+
3,Годунко С. Ю.,1,188.468,До наказу (бюджет),Українська мова та література (ЗНО): 184 Геогр...,—,+
4,Ставничий Н. Т.,2,187.476,До наказу (бюджет),Українська мова та література (ЗНО): 168 Геогр...,—,+
5,Колодежна В. В.,1,187.221,До наказу (бюджет),Українська мова та література (ЗНО): 183 Геогр...,—,+
6,Далудіна Д. В.,2,187.119,До наказу (бюджет),Українська мова та література (ЗНО): 185 Геогр...,—,+
7,Черемшук С. С.,1,186.507,До наказу (бюджет),Українська мова та література (ЗНО): 191 Геогр...,—,+
8,Ольхова І. Е.,1,185.946,До наказу (бюджет),Українська мова та література (ЗНО): 188 Геогр...,—,+
9,Трохимець С. О.,1,185.640,До наказу (бюджет),Українська мова та література (ЗНО): 189 Геогр...,—,+
